In [1]:
import os
import getpass
import streamlit as st
import pickle
import time
import langchain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS

In [2]:
from langchain.chat_models import init_chat_model

In [3]:
if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key :")

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [ ]:
urls = [
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/automobile/tata-motors-unveils-tata-curvv-with-dual-electric-and-ice-versions-article-12789642.html",
    "https://www.moneycontrol.com/automobile/volkswagen-s-1-4-billion-india-tax-tussle-rekindles-foreign-investor-fears-article-12939635.html",
    "https://www.moneycontrol.com/automobile/vw-s-skoda-to-invest-in-manufacturing-evs-in-india-despite-1-4-billion-tax-demand-overhang-article-12965127.html",
]

loader = UnstructuredURLLoader(urls = urls)
data = loader.load()
len(data)

4

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " "],
    chunk_size=1000,
    chunk_overlap=200
)

docs = text_splitter.split_documents(data)
len(docs)

39

In [15]:
# Creating embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vectorindex_google_gemini = FAISS.from_documents(docs, embeddings)
vectorIndex = vectorindex_google_gemini

In [10]:
chain = RetrievalQAWithSourcesChain.from_llm(llm = llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000229FFD48770>, default_metadata=(), model_kwargs={}), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_

In [ ]:
query = "What is the effect of Tesla in Wall street?"

# Number of LLM calls depends on the number of similar chunks which are processed and the answers are combined by making another LLm call
langchain.debug = True

chain({"question" : query},  return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is the effect of Tesla in Wall street?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "T Gnanasekar\n\nprofile\n\nPower Your Trade\n\nprofile\n\nCurrencies with Mecklai Financial\n\nprofile\n\nStock Reports\n\nHomeNewsBusinessMarketsWall Street rises as Tesla soars on AI optimism\n\nTrending Topics\n\nSensex TodayDixon Technologies Share PriceIndusInd Bank Share PriceBEL Share PriceDrone StocksNTPC Green Share Price\n\nWall Street rises as Tesla soars on AI optimism\n\nTesla (TSLA.O) rallied 10% after Morgan Stanley upgraded the electric car maker to \"overweight\" from \"equal-weight,\" saying its Dojo supercomputer could boost the compa

{'answer': 'Tesla\'s stock market performance has a positive effect on Wall Street. On Monday, the Nasdaq closed sharply higher as Tesla\'s stock surged due to optimism surrounding artificial intelligence. Morgan Stanley upgraded Tesla to "overweight," suggesting its Dojo supercomputer could significantly increase the company\'s market value.\n',
 'sources': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}